In [2]:
#Import necesssary libraries

import requests
from urllib.parse import urlencode, urlparse, parse_qsl

import numpy as np
import pandas as pd

In [3]:
#Read the API KEY from the text file
api_file = open("GOOGLE_API_KEY.txt","r")
GOOGLE_API_KEY = api_file.read()
api_file.close()

##### User Inputs (office address & Desired Commute time)

In [4]:
office_address = input("Enter your complete office address: \n") #Get office address as user input


In [5]:
desired_commute_time = int(input("What is your maximum preferred commute time (in mins)? \n"))*60
desired_commute_time

3600

#### Geolocation details of the given office address using Geocode API

In [6]:
data_type = "json"
#def extract_office_info (office_address,data_type='json'):
geocode_base_url = f"https://maps.googleapis.com/maps/api/geocode/{data_type}"
geocode_params = {"address": office_address, "key": GOOGLE_API_KEY}
geocode_url_params = urlencode(geocode_params)
geocode_enpoint_url = f"{geocode_base_url}?{geocode_url_params}"
geocode_response = requests.get(geocode_enpoint_url)
geocode_data = geocode_response.json()['results'][0]
if geocode_response.status_code not in range (200,299):
    {}
latlng = {}
try:
    latlng = geocode_data['geometry']['location']
    office_place_id = geocode_data['place_id']

    for component in geocode_data['address_components']:
        if all(t in component['types'] for t in ['locality']):
            office_locality = component['long_name']
        elif all(t in component['types'] for t in ['administrative_area_level_1']):
            office_state = component['short_name']
        elif all(t in component['types'] for t in ['postal_code']):
            office_postal_code = component['long_name']
        else:
            all(t in component['types'] for t in ['Australia'])
            office_country = component['long_name']
except:
    pass
lat,lng = latlng.get("lat"),latlng.get("lng")
search_text = f"{office_locality} {office_state} {office_country}"
    #return lat,lng, office_place_id,office_postalcode,locality

In [7]:
#Parse locations details (Latitute, Longitude, Postal Code and Suburbs details)
print(f' Below are the details for the entered office location:\n\n Address Entered: {office_address} \n Latitude: {lat},Longitute: {lng}\n Place_id: {office_place_id}\n Suburb: {office_locality} \n Postal_code: {office_postal_code} \n')

 Below are the details for the entered office location:

 Address Entered: 833,collins street, docklands, victoria, australia 
 Latitude: -37.8219417,Longitute: 144.9455252
 Place_id: ChIJH9ixPlld1moRTKcj5LfYyPI
 Suburb: Docklands 
 Postal_code: 3008 



## Getting suburbs using "Find Place"

In [64]:
input_text =  f"suburb near me"
findplace_base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
findplace_params = {
    "key" : GOOGLE_API_KEY,
    "input" : input_text,
    "inputtype" : "textquery",
    "fields" : "place_id,name,geometry,type",
    "locationbias" : f'point:{lat},{lng}',
}
findplace_url_params = urlencode(findplace_params)
findplace_endpoint_url = f'{findplace_base_url}?{findplace_url_params}'
findplace_response = requests.get(findplace_endpoint_url)
findplace_data = findplace_response.json()['candidates']

In [65]:
len(findplace_data)

28

In [67]:
nearby_suburbs =[]
for suburb in findplace_data:
    is_valid = any(t in ['locality'] for t in suburb['types'])
    if is_valid:
        suburb_name = suburb['name']
        suburb_place_id = suburb['place_id']
        place_type = suburb['types']
        
        distance_base_url = f"https://maps.googleapis.com/maps/api/distancematrix/json"
        distance_params = {
        "key" : GOOGLE_API_KEY,
        "origins" : f"place_id:{office_place_id}",
        "destinations" : f"place_id:{suburb_place_id}",
        "mode" : "transit",
        #"transit_mode" : "rail|bus",
        #"transit_routing_preference" : "less_walking"
        #"type" : "locality"
        }
        distance_url_params = urlencode(distance_params)
        distance_endpoint_url = f"{distance_base_url}?{distance_url_params}"
        distance_response = requests.get(distance_endpoint_url)
        distance_data = distance_response.json()
        distance = distance_data['rows'][0]['elements'][0]['distance']['value']
        
        nearby_suburbs.append({
            "Suburb Name" : suburb_name,
            "Place type" : place_type,
            "Distance" : distance
        })

In [69]:
pd.DataFrame(nearby_suburbs)#.sort_values('Distance')

,Suburb Name,Place type,Distance
0,North Melbourne,"[locality, political]",4267
1,Williamstown North,"[locality, political]",22526
2,Burnley,"[locality, political]",6978
3,Princes Hill,"[locality, political]",6464
4,Bellfield,"[locality, political]",15409
5,Balaclava,"[locality, political]",8859
6,Travancore,"[locality, political]",7226
7,West Footscray,"[locality, political]",10681
8,Yarraville,"[locality, political]",9171
9,Aberfeldie,"[locality, political]",14580


# Optional
<details>
  <summary>Click to expand</summary>
  
### Getting Suburbs using "Find Place From Text"
```py
data_type = "json"
#def extract_office_info (office_address,data_type='json'):
geocode_base_url = f"https://maps.googleapis.com/maps/api/geocode/{data_type}"
geocode_params = {"address": "833, Collins Street, Docklands, Victoria, Australia", "key": GOOGLE_API_KEY}
geocode_url_params = urlencode(geocode_params)
geocode_enpoint_url = f"{geocode_base_url}?{geocode_url_params}"
geocode_response = requests.get(geocode_enpoint_url)
geocode_data = geocode_response.json()['results'][0]
if geocode_response.status_code not in range (200,299):
    {}
latlng = {}
try:
    latlng = geocode_data['geometry']['location']
    office_place_id = geocode_data['place_id']

    for component in geocode_data['address_components']:
        if all(t in component['types'] for t in ['locality']):
            office_locality = component['long_name']
        elif all(t in component['types'] for t in ['administrative_area_level_1']):
            office_state = component['short_name']
        elif all(t in component['types'] for t in ['postal_code']):
            office_postal_code = component['long_name']
        else:
            all(t in component['types'] for t in ['Australia'])
            office_country = component['long_name']
except:
    pass
lat,lng = latlng.get("lat"),latlng.get("lng")
search_text = f"{office_locality} {office_state} {office_country}"

input_text =  f"suburb near {search text}"
radius = 55000
findplace_base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
findplace_params = {
    "key" : GOOGLE_API_KEY,
    "input" : input_text,
    "inputtype" : "textquery",
    "fields" : "place_id,name,geometry,type",
    "locationbias" : f"point:{lat},{lng}",
    # "locationbias" : f'circle:{radius}@{lat},{lng}',
}
findplace_url_params = urlencode(findplace_params)
findplace_endpoint_url = f'{findplace_base_url}?{findplace_url_params}'
findplace_response = requests.get(findplace_endpoint_url)
findplace_data = findplace_response.json()['candidates']


nearby_suburbs = []
for suburb in findplace_data:
    desired_types = ['locality']
    is_valid = any(t in desired_types for t in suburb["types"])
    if is_valid:
        suburb_name = suburb["name"]
        suburb_place_id = suburb["place_id"]
        suburb_type = suburb["types"]
        suburb_location = suburb["geometry"]["location"]
        suburb_lat = suburb_location["lat"]
        suburb_lng = suburb_location["lng"]
        
        distance_base_url = f"https://maps.googleapis.com/maps/api/distancematrix/json"
        distance_params = {
        "key" : GOOGLE_API_KEY,
        "origins" : f"place_id:{office_place_id}",
        "destinations" : f"place_id:{suburb_place_id}",
        #"mode" : "transit",
        #"transit_mode" : "rail|bus",
        #"transit_routing_preference" : "less_walking"
        #"type" : "locality"
        }
        distance_url_params = urlencode(distance_params)
        distance_endpoint_url = f"{distance_base_url}?{distance_url_params}"
        distance_response = requests.get(distance_endpoint_url)
        distance_data = distance_response.json()
        try:
            distance = distance_data['rows'][0]['elements'][0]['distance']['value']
        except:
            pass

        nearby_suburbs.append({
                            "Suburb Name" : suburb_name,
                            "Type" : suburb_type,
                            "Distance" : distance 
                            })
len(nearby_suburbs)
```
</details>